In [1]:
# https://simplemaps.com/gis/country/za --> download shapefile

In [3]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pandas as pd
import geopandas as gpd
from utils import primary_dataset
import folium

In [5]:
df = primary_dataset.primary_dataset()

gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)

path_to_data = 'za_shp'
provinces = gpd.read_file(path_to_data, layer='za')
provinces.columns = ['id', 'province', 'source', 'geometry']

We will explore Water Quality over the course of 60 months.


DataSourceError: za_shp: No such file or directory

In [4]:
gdf_with_province = gpd.sjoin(
    gdf_points,
    provinces[["province", "geometry"]],
    how="left",
    predicate="within"
).to_crs("EPSG:4326")

In [5]:
gdf_with_province.head()

,latitude,longitude,sample date,nir,green,swir16,swir22,ndmi,mndwi,pet,total alkalinity,electrical conductance,dissolved reactive phosphorus,month,geometry,index_right,province
0,-28.760833,17.730278,2011-01-02,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,128.912,555.0,10.0,2011-01-31,POINT (17.73028 -28.76083),0.0,Northern Cape
1,-26.861111,28.884722,2011-01-03,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,74.720,162.9,163.0,2011-01-31,POINT (28.88472 -26.86111),6.0,Mpumalanga
2,-26.450000,28.085833,2011-01-03,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,89.254,573.0,80.0,2011-01-31,POINT (28.08583 -26.45),8.0,Gauteng
3,-27.671111,27.236944,2011-01-03,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,82.000,203.6,101.0,2011-01-31,POINT (27.23694 -27.67111),2.0,Free State
4,-27.356667,27.286389,2011-01-03,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,56.100,145.1,151.0,2011-01-31,POINT (27.28639 -27.35667),2.0,Free State


In [ ]:
# plot
zoom_start = 8
m = folium.Map(location=[gdf_with_province['latitude'][0], gdf_with_province['longitude'][1]], tiles="OpenStreetMap", zoom_start=zoom_start)

for coord in zip(gdf_with_province['latitude'], gdf_with_province['longitude']):
    coord = list(coord)

    folium.Marker(
        location=coord,
    ).add_to(m)

m

In [7]:
# !jupyter trust simplemaps.ipynb